In [ ]:
import requests
import pandas as pd 
from sqlalchemy import create_engine , text
from dotenv import load_dotenv
import os

In [ ]:
load_dotenv()

In [3]:
api_data = requests.get("https://fantasy.premierleague.com/api/bootstrap-static/")
json_data = api_data.json()

In [4]:
df = pd.DataFrame(json_data["teams"])

In [ ]:
df.drop(
    columns=["code", "id", "team_division", "unavailable", "pulse_id"],
    inplace=True
)

In [9]:
df.drop(
    columns=["form"],
    inplace=True
)

In [12]:
df.rename(
    columns={
        "name" : "team_name",
        "short_name" : "team_short_name",
        "strength" : "team_strength"
    },
    inplace=True
)

In [ ]:
postgres_URI = os.getenv("POSTGRES_URI") or ""
engine = create_engine(postgres_URI)

In [ ]:
with engine.begin() as conn:
    conn.execute(
        text(
            '''
            create table teams(
                team_id int generated always as identity primary key not null,
                team_name varchar(100) not null,
                draw int ,
                loss int,
                played int,
                points int,
                position int,
                win int,
                team_short_name varchar(50) not null,
                team_strength int, 
                strength_overall_home int ,
                strength_overall_away int, 
                strength_attack_home int ,
                strength_attack_away int,
                strength_defence_home int,
                strength_defence_away int
            );
            '''
        )
    )
    conn.execute(
        text(
            "truncate stg_fixture restart identity"
        )
    )